In [23]:
import pandas as pd
import string

In [24]:
df=pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [25]:
df=df.drop(["Unnamed: 2","Unnamed: 3", "Unnamed: 4"],axis=1)
df.columns=["label","mail"]
df["label"]=df["label"].map({'ham':0,'spam':1})
df.head()

,label,mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
def preprocess(mail):
    #removed the punctuations and lowercased the text
    mail="".join([t.lower() for t in mail if t not in string.punctuation])

    #tokenize
    tokens=mail.split(" ") #tokenize in case of whitespace

    #removing the stopwords hahaha
    return " ".join([t for t in tokens if t not in ENGLISH_STOP_WORDS])

df["mail"]=df["mail"].apply(lambda mail: preprocess(mail))
df.head()




,label,mail
0,0,jurong point crazy available bugis n great wor...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say early hor u c say
4,0,nah dont think goes usf lives
